In [1]:
import numpy as np
import os
import re
import tensorflow as tf
from config import *
import pandas as pd

from tensorflow.python.framework import dtypes
from tensorflow.python.framework import random_seed

/Users/cb.hsu/anaconda/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataPath = "TXF1-Minute-Trade.txt"
rawData = pd.read_csv(dataPath)
print(rawData)

              Date      Time     Open     High      Low    Close  TotalVolume
0        1998/7/22  09:01:00   7950.0   7950.0   7950.0   7950.0            9
1        1998/7/22  09:03:00   7950.0   7950.0   7945.0   7950.0            5
2        1998/7/22  09:05:00   7945.0   7950.0   7945.0   7950.0           13
3        1998/7/22  09:06:00   7950.0   7950.0   7950.0   7950.0            5
4        1998/7/22  09:07:00   7935.0   7950.0   7935.0   7950.0            7
5        1998/7/22  09:08:00   7930.0   7930.0   7930.0   7930.0            2
6        1998/7/22  09:09:00   7910.0   7910.0   7910.0   7910.0            1
7        1998/7/22  09:10:00   7940.0   7940.0   7911.0   7911.0            2
8        1998/7/22  09:12:00   7915.0   7915.0   7915.0   7915.0            1
9        1998/7/22  09:13:00   7915.0   7915.0   7912.0   7912.0            2
10       1998/7/22  09:14:00   7911.0   7911.0   7911.0   7911.0            4
11       1998/7/22  09:15:00   7910.0   7911.0   7910.0   7911.0

In [3]:
offset = 30000
length = 300000
historyLength = 256

In [4]:
close = rawData.Close[offset:offset+length+historyLength].copy().reset_index(drop=True)
print(close)

0         6515.0
1         6513.0
2         6513.0
3         6504.0
4         6503.0
5         6503.0
6         6505.0
7         6501.0
8         6500.0
9         6500.0
10        6495.0
11        6495.0
12        6492.0
13        6495.0
14        6493.0
15        6487.0
16        6486.0
17        6485.0
18        6481.0
19        6482.0
20        6485.0
21        6480.0
22        6490.0
23        6486.0
24        6486.0
25        6490.0
26        6490.0
27        6490.0
28        6485.0
29        6483.0
           ...  
300226    5985.0
300227    5988.0
300228    5986.0
300229    5986.0
300230    5985.0
300231    5983.0
300232    5982.0
300233    5984.0
300234    5984.0
300235    5988.0
300236    5990.0
300237    5988.0
300238    5988.0
300239    5986.0
300240    5986.0
300241    5987.0
300242    5986.0
300243    5992.0
300244    5991.0
300245    5992.0
300246    5988.0
300247    5989.0
300248    5987.0
300249    5988.0
300250    5985.0
300251    5981.0
300252    5978.0
300253    5977

In [5]:
dataSet = rawData.iloc[offset: offset+length+historyLength, 2:2+column].values
print(dataSet)

[[6.515e+03 6.515e+03 6.515e+03 6.515e+03 1.300e+01]
 [6.517e+03 6.517e+03 6.513e+03 6.513e+03 6.000e+00]
 [6.513e+03 6.513e+03 6.513e+03 6.513e+03 3.000e+00]
 ...
 [5.978e+03 5.980e+03 5.975e+03 5.977e+03 9.800e+01]
 [5.976e+03 5.977e+03 5.973e+03 5.976e+03 1.040e+02]
 [5.976e+03 5.980e+03 5.976e+03 5.979e+03 5.900e+01]]


In [6]:
history = np.zeros([length, historyLength*column ])
print(history)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
label = np.zeros([length,2])
print(label)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [8]:
print(close.shape)
print(dataSet.shape)
print(history.shape)
print(label.shape)

(300256,)
(300256, 5)
(300000, 1280)
(300000, 2)


In [9]:
for i in range(historyLength,  length + historyLength) :
    print(i)
    batch = [dataSet[i-historyLength:i, 0:column]]
    batch = np.reshape(batch, [-1])
    #history = np.concatenate((history, batch), axis=0)
    np.copyto(history[i - historyLength], batch)

    #Lable here
    entryPrice = close[i]
    lbl = [0.0, 1.0]
    for futureK in range(i+1, length + historyLength):
        if(close[futureK] - entryPrice > stopWin):
            lbl = [1.0, 0.0]
            break
        elif(close[futureK] - entryPrice < - stopLost):
            lbl = [0.0, 1.0]
            break
    np.copyto(label[i-historyLength],lbl)

256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505


2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256


3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930


5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576


7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
7214
7215


...

294429
294430
294431
294432
294433
294434
294435
294436
294437
294438
294439
294440
294441
294442
294443
294444
294445
294446
294447
294448
294449
294450
294451
294452
294453
294454
294455
294456
294457
294458
294459
294460
294461
294462
294463
294464
294465
294466
294467
294468
294469
294470
294471
294472
294473
294474
294475
294476
294477
294478
294479
294480
294481
294482
294483
294484
294485
294486
294487
294488
294489
294490
294491
294492
294493
294494
294495
294496
294497
294498
294499
294500
294501
294502
294503
294504
294505
294506
294507
294508
294509
294510
294511
294512
294513
294514
294515
294516
294517
294518
294519
294520
294521
294522
294523
294524
294525
294526
294527
294528
294529
294530
294531
294532
294533
294534
294535
294536
294537
294538
294539
294540
294541
294542
294543
294544
294545
294546
294547
294548
294549
294550
294551
294552
294553
294554
294555
294556
294557
294558
294559
294560
294561
294562
294563
294564
294565
294566
294567
294568
294569
294570
294571

295620
295621
295622
295623
295624
295625
295626
295627
295628
295629
295630
295631
295632
295633
295634
295635
295636
295637
295638
295639
295640
295641
295642
295643
295644
295645
295646
295647
295648
295649
295650
295651
295652
295653
295654
295655
295656
295657
295658
295659
295660
295661
295662
295663
295664
295665
295666
295667
295668
295669
295670
295671
295672
295673
295674
295675
295676
295677
295678
295679
295680
295681
295682
295683
295684
295685
295686
295687
295688
295689
295690
295691
295692
295693
295694
295695
295696
295697
295698
295699
295700
295701
295702
295703
295704
295705
295706
295707
295708
295709
295710
295711
295712
295713
295714
295715
295716
295717
295718
295719
295720
295721
295722
295723
295724
295725
295726
295727
295728
295729
295730
295731
295732
295733
295734
295735
295736
295737
295738
295739
295740
295741
295742
295743
295744
295745
295746
295747
295748
295749
295750
295751
295752
295753
295754
295755
295756
295757
295758
295759
295760
295761
295762

296892
296893
296894
296895
296896
296897
296898
296899
296900
296901
296902
296903
296904
296905
296906
296907
296908
296909
296910
296911
296912
296913
296914
296915
296916
296917
296918
296919
296920
296921
296922
296923
296924
296925
296926
296927
296928
296929
296930
296931
296932
296933
296934
296935
296936
296937
296938
296939
296940
296941
296942
296943
296944
296945
296946
296947
296948
296949
296950
296951
296952
296953
296954
296955
296956
296957
296958
296959
296960
296961
296962
296963
296964
296965
296966
296967
296968
296969
296970
296971
296972
296973
296974
296975
296976
296977
296978
296979
296980
296981
296982
296983
296984
296985
296986
296987
296988
296989
296990
296991
296992
296993
296994
296995
296996
296997
296998
296999
297000
297001
297002
297003
297004
297005
297006
297007
297008
297009
297010
297011
297012
297013
297014
297015
297016
297017
297018
297019
297020
297021
297022
297023
297024
297025
297026
297027
297028
297029
297030
297031
297032
297033
297034

298118
298119
298120
298121
298122
298123
298124
298125
298126
298127
298128
298129
298130
298131
298132
298133
298134
298135
298136
298137
298138
298139
298140
298141
298142
298143
298144
298145
298146
298147
298148
298149
298150
298151
298152
298153
298154
298155
298156
298157
298158
298159
298160
298161
298162
298163
298164
298165
298166
298167
298168
298169
298170
298171
298172
298173
298174
298175
298176
298177
298178
298179
298180
298181
298182
298183
298184
298185
298186
298187
298188
298189
298190
298191
298192
298193
298194
298195
298196
298197
298198
298199
298200
298201
298202
298203
298204
298205
298206
298207
298208
298209
298210
298211
298212
298213
298214
298215
298216
298217
298218
298219
298220
298221
298222
298223
298224
298225
298226
298227
298228
298229
298230
298231
298232
298233
298234
298235
298236
298237
298238
298239
298240
298241
298242
298243
298244
298245
298246
298247
298248
298249
298250
298251
298252
298253
298254
298255
298256
298257
298258
298259
298260

299289
299290
299291
299292
299293
299294
299295
299296
299297
299298
299299
299300
299301
299302
299303
299304
299305
299306
299307
299308
299309
299310
299311
299312
299313
299314
299315
299316
299317
299318
299319
299320
299321
299322
299323
299324
299325
299326
299327
299328
299329
299330
299331
299332
299333
299334
299335
299336
299337
299338
299339
299340
299341
299342
299343
299344
299345
299346
299347
299348
299349
299350
299351
299352
299353
299354
299355
299356
299357
299358
299359
299360
299361
299362
299363
299364
299365
299366
299367
299368
299369
299370
299371
299372
299373
299374
299375
299376
299377
299378
299379
299380
299381
299382
299383
299384
299385
299386
299387
299388
299389
299390
299391
299392
299393
299394
299395
299396
299397
299398
299399
299400
299401
299402
299403
299404
299405
299406
299407
299408
299409
299410
299411
299412
299413
299414
299415
299416
299417
299418
299419
299420
299421
299422
299423
299424
299425
299426
299427
299428
299429
299430
299431

In [10]:
np.save("history.npy", history)
np.save("label.npy", label)